In [10]:
import sqlite3
from plate import detect_plate

In [11]:
detect_plate("Car.jpeg")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


d:\PROGRAMS\Projects\ANPR vehicle info\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


'MH20EJ0365'

In [12]:
def get_vehicle_info(plate_number):
    conn = sqlite3.connect('parking_system.db')
    c = conn.cursor()
    
    c.execute('''SELECT v.*, u.* 
                 FROM vehicles v
                 JOIN users u ON v.user_id = u.user_id
                 WHERE v.plate_number = ?''', (plate_number,))
    
    result = c.fetchone()
    conn.close()
    
    if result:
        return {
            'plate': result[0],
            'make': result[1],
            'model': result[2],
            'color': result[3],
            'owner': result[5],
            'contact': result[6],
            'email': result[7],
            'address': result[8]
        }
    return None

In [13]:
def register_user():
    print("\n--- Register New User ---")
    
    user_id = input("Enter user ID: ")
    name = input("Full Name: ")
    contact = input("Contact Number: ")
    email = input("Email Address: ")
    address = input("Physical Address: ")
    
    if not all([user_id, name, contact]):
        print("Error: User ID, Name and Contact are required fields")
        return
    
    try:
        conn = sqlite3.connect('parking_system.db')
        c = conn.cursor()
        
        c.execute("SELECT * FROM users WHERE user_id = ?", (user_id,))
        if c.fetchone():
            print("Error: User ID already exists")
            return
        
        c.execute('''INSERT INTO users 
                    (user_id, name, contact, email, address)
                    VALUES (?, ?, ?, ?, ?)''',
                    (user_id, name, contact, email, address))
        
        conn.commit()
        print(f"Successfully registered user: {name} (ID: {user_id})")
        
        add_vehicle_option = input("Add vehicle for this user now? (y/n): ").lower()
        if add_vehicle_option == 'y':
            add_vehicle(user_id)
            
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()


In [14]:
def add_vehicle(user_id=None):
    print("\n--- Add New Vehicle ---")
    
    if not user_id:
        user_id = input("Enter owner's user ID: ")
    
    conn = sqlite3.connect('parking_system.db')
    c = conn.cursor()
    c.execute("SELECT name FROM users WHERE user_id = ?", (user_id,))
    user = c.fetchone()
    
    if not user:
        print("Error: User not found")
        conn.close()
        return
    
    print(f"Registering vehicle for: {user[0]}")
    plate_number = input("License Plate Number: ").upper().strip()
    make = input("Vehicle Make (e.g., Toyota): ")
    model = input("Vehicle Model (e.g., Corolla): ")
    color = input("Vehicle Color: ")
    
    if not plate_number:
        print("Error: License plate is required")
        conn.close()
        return
    
    try:
        c.execute("SELECT * FROM vehicles WHERE plate_number = ?", (plate_number,))
        if c.fetchone():
            print("Error: This vehicle is already registered")
            return
        c.execute('''INSERT INTO vehicles 
                    (plate_number, make, model, color, user_id)
                    VALUES (?, ?, ?, ?, ?)''',
                    (plate_number, make, model, color, user_id))
        conn.commit()
        print(f"Successfully registered vehicle: {plate_number}")
            
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        conn.close()


In [27]:

def process_camera_image(image_path=None):
    if image_path:
        plate_number = detect_plate(image_path)
    else:
        image_path = input("Enter image path: ")
        plate_number = detect_plate(image_path)
    if plate_number:
        print(f"Detected Plate: {plate_number}")
        vehicle_info = get_vehicle_info(plate_number)
        
        if vehicle_info:
            print("\nVehicle Information:")
            print(f"Make/Model: {vehicle_info['make']} {vehicle_info['model']}")
            print(f"Color: {vehicle_info['color']}")
            print(f"\nOwner Information:")
            print(f"Name: {vehicle_info['owner']}")
            print(f"Contact: {vehicle_info['contact']}")
            print(f"Email: {vehicle_info['email']}")
            
        else:
            print("Vehicle not registered in system.")
    else:
        print("No license plate detected.")

In [28]:
def admin_menu():
    while True:
        print("\nSmart Parking Management System")
        print("1. Register New User")
        print("2. Add Vehicle")
        print("3. Process Camera Image")
        print("4. Exit")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            register_user()
        elif choice == '2':
            add_vehicle()
        elif choice == '3':
            process_camera_image()
        elif choice == '4':
            break
        else:
            print("Invalid choice. exiting.")
            break


In [29]:
admin_menu()


Smart Parking Management System
1. Register New User
2. Add Vehicle
3. Process Camera Image
4. Exit


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
d:\PROGRAMS\Projects\ANPR vehicle info\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Detected Plate: KA64N0099

Vehicle Information:
Make/Model: bmw na
Color: white

Owner Information:
Name: 2
Contact: sr p
Email: 9358870383

Smart Parking Management System
1. Register New User
2. Add Vehicle
3. Process Camera Image
4. Exit


In [20]:
process_camera_image("Car.jpeg")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Detected Plate: MH20EJ0365

Vehicle Information:
Make/Model: Toyota na
Color: black

Owner Information:
Name: 1
Contact: sr
Email: 9358870383
